In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits 
import pickle 

In [2]:
science_list = np.genfromtxt('/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/science/science.list',dtype=str)

In [3]:
science_test_list = science_list[:10] 

In [4]:
median_bias = pickle.load(open("median_bias.p", "rb"))
bias_std = 1.3 # [e] photoelectrons
readout_noise = 7.4  # [e] photoelectrons
gain = 1.91 # [e/ADU]  # this value may be 
median_normalized_flat =  pickle.load(open("median_normalized_flat.p", "rb"))
median_normalized_flat_errors =  pickle.load(open("median_normalized_flat_errors.p", "rb"))

In [5]:
for science_name in science_test_list:
    science_fits = fits.open('/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/science/' + science_name)
    science_data = science_fits[0].data * gain # save the data from the first HDU 
    science_fits.close()

    science_debiased = science_data - median_bias
    science_corrected = science_debiased / median_normalized_flat

    ## Error associated to the science corrected frame
    science_debiased_errors = np.sqrt(readout_noise**2 + bias_std**2 + science_debiased)
    science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)

/tmp/ipykernel_2829/1292183735.py:7: RuntimeWarning: divide by zero encountered in divide
  science_corrected = science_debiased / median_normalized_flat
/tmp/ipykernel_2829/1292183735.py:7: RuntimeWarning: invalid value encountered in divide
  science_corrected = science_debiased / median_normalized_flat
/tmp/ipykernel_2829/1292183735.py:11: RuntimeWarning: divide by zero encountered in divide
  science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)
/tmp/ipykernel_2829/1292183735.py:11: RuntimeWarning: invalid value encountered in multiply
  science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)


In [6]:
print(science_name)
print(science_name[:-5])
new_name = science_name[:-5] + '_corr.p'
print(new_name)

AF802694.fits
AF802694
AF802694_corr.p


In [7]:
for science_name in science_test_list:
    science_fits = fits.open('/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/science/' + science_name)
    science_data = science_fits[0].data * gain # save the data from the first HDU 
    science_fits.close()
    
    science_debiased = science_data - median_bias
    science_corrected = science_debiased / median_normalized_flat

    ## Error associated to the science corrected frame
    science_debiased_errors = np.sqrt(readout_noise**2 + bias_std**2 + science_debiased)
    science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)

#Here put every task like centroid determination, jd to bjd, aperture photometry


    new_name = '/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/correct/' + science_name[:-5] + '_corr.p'
    pickle.dump(science_corrected, open(new_name, 'wb'))
    new_name = '/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/correct/' + science_name[:-5] + '_corr_errors.p'
    pickle.dump(science_corrected_errors, open(new_name, 'wb'))

/tmp/ipykernel_2829/2045413223.py:7: RuntimeWarning: divide by zero encountered in divide
  science_corrected = science_debiased / median_normalized_flat
/tmp/ipykernel_2829/2045413223.py:7: RuntimeWarning: invalid value encountered in divide
  science_corrected = science_debiased / median_normalized_flat
/tmp/ipykernel_2829/2045413223.py:11: RuntimeWarning: divide by zero encountered in divide
  science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)
/tmp/ipykernel_2829/2045413223.py:11: RuntimeWarning: invalid value encountered in multiply
  science_corrected_errors = science_corrected * np.sqrt((science_debiased_errors/science_debiased)**2 + (median_normalized_flat_errors/median_normalized_flat)**2)


In [8]:
n_images = len(science_test_list)
array_jd = np.zeros(n_images)
array_exptime = np.zeros(n_images)
array_airmass = np.zeros(n_images)

for i_science, science_name in enumerate(science_test_list):
    science_fits = fits.open('/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/science/' + science_name)
    array_jd[i_science] = science_fits[0].header['JD']
    array_airmass[i_science] = science_fits[0].header['AIRMASS']
    array_exptime[i_science] = science_fits[0].header['EXPTIME']
    science_fits.close()

print(array_jd)
print(array_airmass)
print(array_exptime)

[2459958.325123 2459958.325215 2459958.325306 2459958.325399
 2459958.32549  2459958.325582 2459958.325674 2459958.325767
 2459958.32586  2459958.325952]
[1.186 1.186 1.186 1.186 1.185 1.185 1.185 1.184 1.184 1.184]
[6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]


mean sun = fictious "fake" constant speed on the equator during the year. real sun and mean sun has to match when passing to the gamma point
gamma point = intersection of the ecliptic with equator
mean sun is used to call the universal time UT
mean solar time = hour angle HA of mean sun + 12 (perchè siamo a mezzanotte ma vogliamo sia 0 e non 12)
ST sidereal time = RA right ascension + HA hour angle (HA of the gamma point)
UT is a mean solar time at greenwhich
UTC universinal cordinated time where the duration of a second it is fixed using the atome international time TAI
difference between 86400 s in tai and mean solar time (UT AND UTC have different durations of a second). this difference must be taken into account when observing.
1 problem) UTC discontinuous
2 problem) UTC in on earth (affected by gravity) (it is a proper time)

we use BJD_TDB (times du baricenter, dynamical time to baricenter) (cordinated time) time corrected by gravity (for discontinuities), for space telescopes too. referenced to the baricenter system of sun and earth

another complication after correcting from UTC to BJD_TDB is that if you observe from a point of the orbit and next time on the opposite point, you have to take into account that the time the light take to travel to the second position is more (60 minutes). so we need to compute the light travel time like it is a correction to the BJD_TDB to bring the time to the center of baricenter of the orbit

JD indicates the location (earth), UTC define the scale you define for measure the time (utc standard use fixed duration of the second + 1 second something like that). We need to convert it to BJD_TDB (B is baricentered or baricentric in the solar systemm for BJD and keeping into account the ligh travel of photon for TDB)

conversion from jd to bjd

In [9]:
array_midexposure = array_jd + array_exptime/2. /86400.
print(array_midexposure)

[2459958.32515772 2459958.32524972 2459958.32534072 2459958.32543372
 2459958.32552472 2459958.32561672 2459958.32570872 2459958.32580172
 2459958.32589472 2459958.32598672]


now from jd_utc to bjd_tdb

In [10]:
from astropy.time import Time

#we need the coordinates of the target
from astropy import coordinates as coord
from astropy import units as u

AttributeError: module 'numpy._core.umath' has no attribute '_ljust'

In [ ]:
science_fits = fits.open('/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/science/' + science_test_list[0])
print(science_fits[0].header['RA'], science_fits[0].header['DEC'])
print(science_fits[0].header['OBJCTRA'], science_fits[0].header['OBJCTDEC'])

In [ ]:
telescope = coord.SkyCoord('06:30:31.57', '+29:40:23', unit=(u.hourangle, u.deg), frame='icrs')
target = coord.SkyCoord('06:30:32.79', '+29:40:20', unit=(u.hourangle, u.deg), frame='icrs')

In [ ]:
print(telescope)
print(target)
print(telescope.ra) #esempio printando solo ra
print(telescope.ra.deg) #esempio printando ra in deg

In [ ]:
print(science_fits[0].header['SITELAT'])
print(science_fits[0].header['SITELONG'])

In [ ]:
time_object = Time(array_midexposure, format='jd', scale='utc', location=('45.8472d', '11.567d')) #location of the telescope

In [ ]:
bjd_tdb = time_object.tdb + time_object.light_travel_time(target, ephemeris='jpl')

In [ ]:
print(bjd_tdb)
print((bjd_tdb.value - array_midexposure)*86400) #baricenter and light travel time difference from the jd_utc

In [ ]:
jd_plot = np.arange(2460000.000, 2460365.25, 0.01)
time_plot = Time(jd_plot, format='jd', scale='utc', location=('45.8472d', '11.567d'))
ltt_target_plot = time_plot.light_travel_time(target, ephemeris='jpl')
ltt_telescope_plot = time_plot.light_travel_time(telescope, ephemeris='jpl')

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(jd_plot, ltt_target_plot.to_value(u.min), label ='LTT of target')
plt.plot(jd_plot, ltt_telescope_plot.to_value(u.min), label ='LTT of telescope')
plt.legend()
plt.xlabel('JD')
plt.ylabel('Minutes')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(jd_plot, ltt_target_plot.to_value(u.s) - ltt_telescope_plot.to_value(u.s), label ='LTT difference')
plt.legend()
plt.xlabel('JD')
plt.ylabel('Minutes')
plt.show()

In [ ]:
science_frame_name = '/home/alcadis/Desktop/lab2/group13_WASP-12_20230113/correct/' + science_test_list[-1][:-5] + '_corr.p'
science_corrected = pickle.load(open(science_frame_name, 'rb'))
import matplotlib.colors as colors

vmin = np.amin(science_corrected[:,100:400])
vmax = np.amax(science_corrected[:,100:400])
print('vmin: {0:1.2f} vmax: {1:5.2f}' .format(vmin, vmax))

fig, ax = plt.subplots(1, figsize=(8,3))
im1 = plt.imshow(science_corrected, cmap=plt.colormaps['magma'], norm=colors.LogNorm(vmin=vmin, vmax=vmax), origin='lower')
plt.colorbar(im1, ax=ax)
plt.xlabel('X [pixels]')
plt.ylabel('Y [pixels]')
plt.show()